In [ ]:
#################################################################### Loading packages ####################################################################
import sys as sys
import os
import numpy as np
import scipy.stats as scs
from matplotlib import pyplot as plt
from mpl_toolkits.basemap import Basemap
from tqdm import tqdm
import pickle
import math 
import re
from Censored_copula import *
from scipy.spatial import distance_matrix
from sklearn.metrics.pairwise import euclidean_distances 
import pandas as pd
from scipy.optimize import dual_annealing
from joblib import Parallel, delayed
import psutil
from scipy.spatial.distance import pdist
from sklearn.metrics import pairwise_distances
from sklearn.gaussian_process.kernels import Matern
plt.rcParams.update({'font.size': 30})

#################################################################### Loading GNM ####################################################################
model_version = 0 
target_distribution = "gamma_hurdle"
base_nn = "HConvLSTM_tdscale"
sys.path.append('../')

import pickle
import torch

path_ = f"Checkpoints/uk_rain_DGLM_{base_nn}_{target_distribution}/lightning_logs/version_{model_version}/"
#ckpt_path = glms.NeuralDGLM.get_ckpt_path(os.path.join(path_,"checkpoints"))
print(path_)
GNM_output = pickle.load( open(os.path.join(path_,".pkl"),"rb") )

#################################################################### Problematic locations? ###############################################################

location_keys = list(GNM_output.keys())                 # '([0, 16], [44, 60])', ...
output_keys = list(GNM_output[location_keys[0]].keys()) # 0'pred_mu', 1'pred_disp', 2'target_did_rain', 3'target_rain_value', 4'date', 5'pred_p', 6'mask', 7'idx_loc_in_region'


# May have to remove a location that doesn't have the correct number of days:
#location_keys.remove(location_keys[344]) #to remove for gamma_20_20
#location_keys.remove(location_keys[171]) # to remove for gamma_20_20_unseen
#location_keys.remove(location_keys[344]) # to remove for 10 y train
#location_keys.remove(location_keys[312])  # to remove for 2y_test
#location_keys.remove(location_keys[336]) #to remove for gamma_20_20 train  
#################################################################### Utility functions ####################################################################
def give_lat_lon(location_key):
    return [ int(re.findall(r'\d+',string)[0]) for string in location_key.split(',')]

def is_loc_land(location,row,column):
    return GNM_output[location]['mask'][0][row][column]

#location_keys.remove('lat_50.95_49.35_lon_-4.55_-2.95') # 'lat_50.95_49.35_lon_-4.55_-2.95' has no data after week indexed 255, so I did not consider it.
day_dates = [pd.to_datetime(i).date() for i in GNM_output[location_keys[0]]['date']]
output_keys.remove('idx_loc_in_region') # not usefull
output_keys.remove('date') # no more usefull

#################################################################### Making list for land locations ####################################################################
#GNM_landonly[location][param][day]
# 0'pred_mu', 1'pred_disp', 2'target_did_rain', 3'target_rain_value', 4'pred_p', 5'location'

GNM_landonly = [] 
land_counter = -1
for location in tqdm(location_keys):
    for col in range(4):
        for row in range(4):
            if not is_loc_land(location,row,col):#check if land
               continue  
            land_counter+=1
            GNM_landonly.append([[],[],[],[],[]]) # parameters except location
            temp_lat_lon = give_lat_lon(location)
            GNM_landonly[land_counter].append([temp_lat_lon[1]-8+row,(temp_lat_lon[2])+6+col]) #location
            for param_idx,param in enumerate(output_keys[:-1]):
                for day in range(len(GNM_output[location_keys[0]]['pred_mu'])):
                    qqq = [location,param,day,row,col]
                    GNM_landonly[land_counter][param_idx].append(GNM_output[location][param][day][row,col])

dist_mat = distance_matrix(np.matrix([[GNM_landonly[loc][5][0],GNM_landonly[loc][5][1]] for loc in range(len(GNM_landonly))]),np.matrix([[GNM_landonly[loc][5][0],GNM_landonly[loc][5][1]] for loc in range(len(GNM_landonly))]))

#################################################################### Elevation data ####################################################################
import netCDF4 as nc
fn = 'topo_0.1_degree.nc'
ds = nc.Dataset(fn)
h = ds['Band1'][:]
h = np.flip(np.matrix(h),axis=0)
topo_dist = euclidean_distances(np.array([h[loc[5][0],loc[5][1]+2] for loc in GNM_landonly]).reshape(-1,1))

In [ ]:
for i,loc in enumerate(location_keys):
    if loc=='([80, 96], [64, 80])':
        print(i,loc)

### Plotting to make sure everything is in order

In [ ]:
#check rain is correct
for day in range(10,11):
    fig = plt.gcf() 
    fig.set_size_inches(14,10)

    plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(GNM_landonly))],[-GNM_landonly[loc][5][0] for loc in range(len(GNM_landonly))],c=[GNM_landonly[loc][0][day] for loc in range(len(GNM_landonly))],cmap='inferno_r',s=27,marker='s')
    plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
    plt.show()

## Forecasts

Has to be done with the train set, so remember to change the JGNM experiment.

### Converting observations to us + Sample covariance matrix?

In [ ]:
#GNM_landonly[location][param][day]
# 0'pred_mu', 1'pred_disp', 2'target_did_rain', 3'target_rain_value', 4'pred_p', 5'location'
def get_cdf_gnm(places):
    out=[]
    for idx,place in enumerate(places):
        out.append([])    
        for day in range(len(place[3])):
            scaling = np.random.uniform(size=1)
            obs = place[3][day]
            p = place[4][day]

            if obs==0: # dry day -> 1-p
                out[idx].append(scaling*(1-p))
            else: # rainy day -> (1-p)+p.cdf_x
                mu,disp = place[0][day],place[1][day]
                out[idx].append( (1-p)+(p*scs.gamma(scale=disp*mu,a=1/disp).cdf(obs)) )
    return out

obs_u_all_ = []
for locs in tqdm(np.array_split(np.array(range(len(GNM_landonly))),67)):
    
    size = math.ceil(len(locs)/psutil.cpu_count())
    locs_chuncks = [[GNM_landonly[i] for i in locs][x:x+size] for x in range(0,len(locs), size)]
    
    obs_u_all_.append( Parallel(n_jobs=psutil.cpu_count())(delayed(get_cdf_gnm)(chunck) for chunck in locs_chuncks) )



In [ ]:
# Save
with open('obs_u_all_.txt','wb') as f:
    pickle.dump(obs_u_all_,f)
'''
#Load 
with open('obs_u_all_.txt','rb') as f:
    obs_u_all_ = pickle.load(f)'''

In [ ]:
plt.figure(figsize=(20,5))
obs_u_all = np.concatenate(np.array(obs_u_all_).flatten()).flatten()
obs_u_all = np.array(obs_u_all).astype(float)
plt.scatter( range(len(obs_u_all)),obs_u_all,s=0.000005)
plt.show()

### Difference between adding and multiplying kernels

In [ ]:
from sklearn.gaussian_process.kernels import Matern
M_topo,M_dist = Matern(length_scale=450,nu=3.5).__call__(topo_dist) , Matern(length_scale=450,nu=3.5).__call__(dist_mat) 
test_multiply =np.multiply( M_topo,M_dist)
test_add =np.add((0.1/70) *M_topo,0.9*M_dist )

In [ ]:
plt.matshow(M_topo)
fig = plt.gcf() 
fig.set_size_inches(14,10)
plt.colorbar()

plt.matshow(M_dist)
fig = plt.gcf() 
fig.set_size_inches(14,10)
plt.colorbar()

plt.matshow(test_add)
fig = plt.gcf() 
fig.set_size_inches(14,10)
plt.colorbar()

cov_rain = np.cov(np.transpose(obs_norm_all))
plt.matshow(cov_rain)
fig = plt.gcf() 
fig.set_size_inches(14,10)
plt.colorbar()

In [ ]:

plt.matshow(np.multiply(topo_dist[:2000,:2000],dist_mat[:2000,:2000]),cmap='viridis_r',vmax=13000)
plt.colorbar()
fig = plt.gcf() 
fig.set_size_inches(14,10)
plt.matshow(topo_dist[:2000,:2000],cmap='viridis_r')
plt.colorbar()

fig = plt.gcf() 
fig.set_size_inches(14,10)
plt.matshow(dist_mat[:2000,:2000],cmap='viridis_r')
plt.colorbar()
fig = plt.gcf() 
fig.set_size_inches(14,10)

### Finding a local set of points

In [ ]:
# finds the indexes (gives a nested array).Add [0] to get it as an array.
np.where([math.dist(GNM_landonly[k][5],GNM_landonly[10][5])<2 for k in range(len(GNM_landonly))])[0]
# gives the loctions themselves
[GNM_landonly[i][5] for i in np.where([math.dist(GNM_landonly[k][5],GNM_landonly[10][5])<2 for k in range(len(GNM_landonly))])[0]]

In [ ]:
# spacially subsampling
fig = plt.gcf() 
fig.set_size_inches(14,10)
random_center = 3531
locs = np.random.choice(np.where([math.dist(GNM_landonly[k][5],GNM_landonly[random_center ][5])<15 for k in range(len(GNM_landonly))])[0],400,replace=False)
print(len(locs))

plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(GNM_landonly))],
[-GNM_landonly[loc][5][0] for loc in range(len(GNM_landonly))],
color='grey',s=27,marker='s')

plt.scatter([GNM_landonly[loc][5][1] for loc in locs],
[-GNM_landonly[loc][5][0] for loc in locs],
c=[GNM_landonly[loc][3][day] for loc in locs],
cmap='inferno_r',s=27,marker='s')

plt.xlim(0,140)
plt.ylim(-100,0)
plt.show()

In [ ]:
def generate_subsample_locs(locs_nb):
    locs=[]
    while len(locs)<locs_nb:
        #random_center = np.random.choice(range(len(GNM_landonly))) 
        random_center = 3532  
        locs = np.where([math.dist(GNM_landonly[k][5],GNM_landonly[random_center ][5])<(np.sqrt(locs_nb/np.pi)+3) for k in range(len(GNM_landonly))])[0]
    return np.random.choice(locs,size=locs_nb,replace=False)    

In [ ]:
# Alternative: make sure to get the same number of locs.

# spacially subsampling
fig = plt.gcf() 
fig.set_size_inches(14,10)
locs = generate_subsample_locs(locs_nb=400)
print(len(locs))

plt.scatter([GNM_landonly[loc][5][1] for loc in locs],
[-GNM_landonly[loc][5][0] for loc in locs],
c=[GNM_landonly[loc][3][300] for loc in locs],
cmap='inferno_r',s=27,marker='s')

plt.xlim(0,140)
plt.ylim(-100,0)
plt.show()

In [ ]:
for i,v in enumerate(np.array([GNM_landonly[l][5] for l in range(len(GNM_landonly))])): # closest to center 1107 [43 70]

    if math.dist(v,[74,107])<2:
        print(i,v)

### Estimation with Scoring Rules

In [ ]:
# Transform observations into x with respect to their marginals and then to a normal scale. (1-p) + p*CDF(x), 1m30

obs_norm_all = []
for day in tqdm(range(len(GNM_landonly[0][1]))):
    
    # Transform observations into Us with respect to their marginals. (1-p) + p*CDF(x)
    us = np.array(1-np.array([GNM_landonly[l][4][day] for l in range(len(GNM_landonly))]) + np.multiply( # 1-p +
                        [GNM_landonly[l][4][day] for l in range(len(GNM_landonly))],                     # p *
                        scs.gamma(                                                          # CDF(x)
                            scale=np.multiply([GNM_landonly[l][0][day] for l in range(len(GNM_landonly))],[GNM_landonly[l][1][day] for l in range(len(GNM_landonly))]), # scale = mu * disp
                            a=np.reciprocal([GNM_landonly[l][1][day] for l in range(len(GNM_landonly))])                                                   # a = shape = 1/disp 
                        ).cdf([GNM_landonly[l][3][day] for l in range(len(GNM_landonly))])               # x : observed rain
                    ))

    # Make sure no Us are above the precision of norm.ppf. Fix them to the next highest value of that day.
    if np.sum([us>=1-(1e-15)])>0:
        us[us>1]=np.max(us[us<1-(1e-15)])

    # Transform Us into normal variables.
    obs_norm_all.append(
        scs.norm.ppf(us)
    )
obs_norm_all = np.nan_to_num(obs_norm_all, copy=True, nan=8.5, posinf=8.5, neginf=-8.5)

censor_levels_di = []

for day in tqdm(range(len(GNM_landonly[0][1]))):
    
    # Transform observations into Us with respect to their marginals. (1-p) + p*CDF(x)
    ui = np.array(1-np.array([GNM_landonly[l][4][day] for l in range(len(GNM_landonly))])) # 1-p

    #print(ui)

    # Transform Us into normal variables.
    censor_levels_di.append(
        scs.norm.ppf(ui)
    )

censor_levels_di = np.array(censor_levels_di)


In [ ]:
def generate_subsample_locs(location_number):
    l=[]
    while len(l)<location_number:
        random_center = np.random.choice(range(len(GNM_landonly))) 
        l = np.where([math.dist(GNM_landonly[k][5],GNM_landonly[random_center ][5])<(np.sqrt(location_number/np.pi)+30) for k in range(len(GNM_landonly))])[0]
    return np.random.choice(l,size=location_number,replace=False)    

subset_locations = []
for s in range(250):
    subset_locations.append(generate_subsample_locs(location_number=800))

In [ ]:
# SR estimates

def Spatial_Energy_Score_add_first(theta,locs_nb,days_nb,sim_nb,observations_x,censor_levels_di,dist_matrix,topo_matrix=None):
    '''
    theta: vector of 3 [Matern_param, coef_distmat, coef_topomat]. The coefs are applied to the matricies to form the ditance matrix used as input in the Matern kernel to form the covariance.
    locs_subsets: subsets of locations with the same number of locs in each.
    days_nb: how many days to include on the subsample.
    sim_nb: number of simulation draws to compare with original data in the score.
    observations_x: x, observed rainfall transformed to a Gaussian normal scale. This is the data that we try to emulate.
    censor_levels_di: d, probability of no rain (1-p) transformed to a normal scale. These are used to censor simulations (x') when comparingt to x.
    '''
    # Subsample locations. Choose a random snippet from the subsets in locs_subsets.
        # OLD: Subsample locations. Choose a random center and take all locations within fixed radius.
    '''random_center = np.random.choice(range(len(GNM_landonly))) 
    locs = np.where([math.dist(GNM_landonly[k][5],GNM_landonly[random_center][5])<radius for k in range(len(GNM_landonly))])[0]
    locs_nb = len(locs)'''
        # New: pick one of the lists of location indexes from the locs_subsets provided.
    locs = generate_subsample_locs(locs_nb)

    # Subsample days.
    days = np.random.choice(range(len(GNM_landonly[0][0])),size=days_nb,replace=False)

    # Select relevant subsamples of data.
    x = observations_x[days,:][:,locs]
    d = censor_levels_di[days,:][:,locs]
    
    # Create covariance matrix.
    if np.sum(topo_matrix==None):
        cov_mat = np.nan_to_num(Matern(length_scale=theta[0],nu=3.5).__call__(dist_matrix[locs,:][:,locs]))
    else:
        cov_mat = np.nan_to_num(Matern(length_scale=theta[0],nu=3.5).__call__(theta[1]*dist_matrix[locs,:][:,locs]+topo_matrix[locs,:][:,locs]))
    
    # Draw samples from the multivariate nromal with appropriate covariance.
    simulations = scs.multivariate_normal(mean=np.zeros(locs_nb),cov=cov_mat,allow_singular=True).rvs(size=sim_nb)
    
    # Censor simulated values and format them into [[sim0-day0,sim0-day1,...,sim0-dayn] ,..., [sim_m-day0,sim_m-day1,...,sim_m-dayn]]. Gives sim_nb*days_nb=m*n vectors of len=locs_nb. 
    censored_sims = np.maximum(np.repeat(simulations,days_nb,axis=0),np.tile(d,(sim_nb,1)))

    # ||x' - x' ||_2 component of energy score
    sims_by_day = [censored_sims[0+k::days_nb] for k in range(days_nb)] # regroup to get [[sim(1,day1), sim(2,day1),...,sim(m,day1)],...,[sim(1,dayn), sim(2,dayn),...,sim(m,dayn)]].
    
    sim_L2_diff = 0
    
    for sim_day_i in sims_by_day: # don't know how to make it more efficient... maybe can parallelize to compute complete score
        
        # ||x' - x' ||_2 component of energy score
            # OLD: einsum from Lorenzos code on each sim_day_i was 9x slower then doing pdist on each sim_day_i. Both give the same answer
        '''
        # matrix of all combinations over i of [vector_i-vector_1, ... , vector_i-vector_n]
        temp_diff_combinations_matrix = sim_day_i.reshape(1,sim_nb,locs_nb)-sim_day_i.reshape(sim_nb,1,locs_nb) 
        # einsum creates a matrix of sum of squares, then take sqrt to get L2.
        sim_L2_diff+=0.5*np.sum(np.sqrt(np.einsum('ijk, ijk -> ij',temp_diff_combinations_matrix,temp_diff_combinations_matrix))) 
        '''
            # New:
        sim_L2_diff+=np.sum(pdist(sim_day_i))

    # Add both parts of Energy.
    Energy = np.mean(np.linalg.norm(censored_sims - np.tile(x,(sim_nb,1)),axis=1)) - sim_L2_diff/(days_nb*sim_nb*(sim_nb-1))
    
    out =2* Energy 
    path_.append([theta,out])
    print([theta,out])
    #print(theta,out)
    return out

def Spatial_Energy_Score_sublocs(theta,beta,sub_locs,days_nb,sim_nb,observations_x,censor_levels_di,dist_matrix,topo_matrix=None):
    '''
    theta: vector of 3 [Matern_param, coef_distmat, coef_topomat]. The coefs are applied to the matricies to form the ditance matrix used as input in the Matern kernel to form the covariance.
    sub_locs: a set of subsets of location indexes with the same number of locs in each.
    days_nb: how many days to include on the subsample.
    sim_nb: number of simulation draws to compare with original data in the score.
    observations_x: x, observed rainfall transformed to a Gaussian normal scale. This is the data that we try to emulate.
    censor_levels_di: d, probability of no rain (1-p) transformed to a normal scale. These are used to censor simulations (x') when comparingt to x.
    '''
    # Subsample locations. Choose a random subset the list of subsets - sub_locs.
    locs = sub_locs[np.random.choice(range(len(sub_locs)))] 
    locs_nb = len(locs)
    # Subsample days.
    days = np.random.choice(range(len(observations_x)),size=days_nb,replace=False)

    # Select relevant subsamples of data.
    x = observations_x[days,:][:,locs]
    d = censor_levels_di[days,:][:,locs]
    
    # Create covariance matrix.
    if np.sum(topo_matrix==None):
        cov_mat = np.nan_to_num(Matern(length_scale=theta,nu=3.5).__call__(dist_matrix[locs,:][:,locs]))
    else:
        cov_mat = np.nan_to_num(Matern(length_scale=theta[0],nu=3.5).__call__(theta[1]*dist_mat[locs,:][:,locs]+((1-theta[1])/70)*topo_dist[locs,:][:,locs]))

    # Draw samples from the multivariate nromal with appropriate covariance.
    simulations = scs.multivariate_normal(mean=np.zeros(locs_nb),cov=cov_mat,allow_singular=True).rvs(size=sim_nb)
    
    # Censor simulated values and format them into [[sim0-day0,sim0-day1,...,sim0-dayn] ,..., [sim_m-day0,sim_m-day1,...,sim_m-dayn]]. Gives sim_nb*days_nb=m*n vectors of len=locs_nb. 
    censored_sims = np.maximum(np.repeat(simulations,days_nb,axis=0),np.tile(d,(sim_nb,1)))

    # ||x' - x' ||_2 component of energy score
    sims_by_day = [censored_sims[0+k::days_nb] for k in range(days_nb)] # regroup to get [[sim(1,day1), sim(2,day1),...,sim(m,day1)],...,[sim(1,dayn), sim(2,dayn),...,sim(m,dayn)]].
    
    sim_L2_diff = 0
    
    for sim_day_i in sims_by_day: # don't know how to make it more efficient... maybe can parallelize to compute complete score
        
        # ||x' - x' ||_2 component of energy score
            # OLD: einsum from Lorenzos code on each sim_day_i was 9x slower then doing pdist on each sim_day_i. Both give the same answer
        '''
        # matrix of all combinations over i of [vector_i-vector_1, ... , vector_i-vector_n]
        temp_diff_combinations_matrix = sim_day_i.reshape(1,sim_nb,locs_nb)-sim_day_i.reshape(sim_nb,1,locs_nb) 
        # einsum creates a matrix of sum of squares, then take sqrt to get L2.
        sim_L2_diff+=0.5*np.sum(np.sqrt(np.einsum('ijk, ijk -> ij',temp_diff_combinations_matrix,temp_diff_combinations_matrix))) 
        '''
            # New: pdist takes the offdiagonal of an L2 distance matrix, only counting once each combination of Y,Y'.
        sim_L2_diff+= 2 * np.sum(np.power(pdist(sim_day_i),beta))

    # Add both parts of Energy.
    Energy = 2 * np.mean(np.power(np.linalg.norm(censored_sims - np.tile(x,(sim_nb,1)),axis=1),beta)) - sim_L2_diff/(days_nb*sim_nb*(sim_nb-1))
    
    
    path_.append([theta,Energy])
    print([theta,Energy])

    return Energy

path_avg = []

def Spatial_Energy_Score_sublocs_averaged(theta,beta,sub_locs,days_nb,sim_nb,observations_x,censor_levels_di,dist_matrix,topo_matrix,avg_nb):
    avg_SR = 0

    for s in range(avg_nb):
        avg_SR+= Spatial_Energy_Score_sublocs(theta,beta,sub_locs,days_nb,sim_nb,observations_x,censor_levels_di,dist_matrix,topo_matrix)

    path_avg.append(avg_SR/avg_nb)
    
    return avg_SR/avg_nb

In [ ]:
path_=[]
path_avg = []
Spatial_Energy_Score_sublocs_averaged(theta=[800,0.9],
                                      beta=0.2, 
                                      sub_locs=subset_locations, 
                                      days_nb=700,sim_nb=150, 
                                      observations_x=obs_norm_all, 
                                      censor_levels_di=censor_levels_di, 
                                      dist_matrix=dist_mat,
                                      topo_matrix=topo_dist,
                                      avg_nb=4) 

In [ ]:
# Optimiser

path_ = []
path_avg = [] # 100 sets of 800 locs, 700 days, 150 sims, beta=0.2 , avg of 4 =>25s/SRavg

# In the case of the 2y robustness, 700 is all days in dataset, as len(GNM_landonly[0][0])=700 => only variation should come from subsampling locs and the sims themselves.
dual_annealing(Spatial_Energy_Score_sublocs_averaged,[(100,1500),(0.5,0.99)],args=(0.2,subset_locations,700,150,obs_norm_all,censor_levels_di,dist_mat,topo_dist,4),no_local_search=True,x0=[310,0.9],maxiter=13000)

In [ ]:
'''# Save
with open('path_.txt','wb') as f:
    pickle.dump(path_,f)
'''
#Load 
with open('path_.txt','rb') as f:
    path_ = pickle.load(f)

In [ ]:
fig=plt.gcf()
plt.figure(figsize=(10,7))
plt.scatter(range(len(path_)),path_[:,1],s=1)



In [ ]:
np.argmin(path_[:,1])

In [ ]:
fig = plt.gcf() 
fig.set_size_inches(10,10)
plt.hist(path_[:,1],bins=50)
plt.title('SR')
plt.tight_layout()

plt.show()

fig = plt.gcf() 
fig.set_size_inches(10,10)
plt.tight_layout()
plt.hist(path_[:,0],bins=50)
plt.title('l_scale')

In [ ]:
fig = plt.gcf() 
fig.set_size_inches(10,10)
plt.scatter(path_[:,0],path_[:,1],s=1)
plt.title('l_scale vs SR')
plt.tight_layout()
plt.show()


#### KNN averaging for SR estimates

In [ ]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')
knn.fit(path_[:,0].reshape(-1,1))


In [ ]:
fig = plt.gcf() 
fig.set_size_inches(10,10)
plt.tight_layout()
for l in np.linspace(350,800,200):
    plt.scatter(l,    np.mean([path_[c,1] for c in knn.kneighbors(np.array([[l]]),return_distance=False,n_neighbors=400)]),s=50)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 30})
fig = plt.gcf() 
fig.set_size_inches(15,10)
l_scales = [np.mean(
    [path_[c,1] for c in knn.kneighbors(np.array([[l]]),return_distance=False,n_neighbors=300)])
     for l in np.linspace(200,1000,500)]

plt.scatter( np.linspace(200,1000,500) , l_scales ,color='orange',s=30,label='Energy Score Estimates')
plt.plot([450,450],[1.185,1.22],color='black',linestyle='dotted',linewidth=3,label=r'True $\theta ^{*}$ = 450')

plt.legend()
plt.tight_layout()
plt.xlabel(r'$\theta$')
plt.ylabel('Energy Score')
plt.savefig('SR_lscale_fakedata.eps', format='eps')
plt.show()


In [ ]:
l_scales = np.linspace(200,1500,50)
a_coefs = np.linspace(0.5,1,50)
L,A = np.meshgrid(l_scales,a_coefs)
Z=np.zeros(L.shape)
Z_variance=np.zeros(L.shape)
for l in tqdm(range(Z.shape[0])):
    for a in range(Z.shape[1]):
        Z[l,a] = np.mean([np.array([path_[k][1] for k in range(len(path_))])[k] for k in knn.kneighbors(np.array([[L[l,a],A[l,a]]]),return_distance=False,n_neighbors=300)])
        Z_variance[l,a] = np.std([np.array([path_[k][1] for k in range(len(path_))])[k] for k in knn.kneighbors(np.array([[L[l,a],A[l,a]]]),return_distance=False,n_neighbors=300)])

In [ ]:
fig = plt.gcf() 
fig.set_size_inches(10,10)
plt.contourf(L,A,(Z),250,cmap='turbo')
plt.colorbar()
plt.scatter(450,0.9,s=200,c='red',label='True theta: [450,0.9]')
plt.legend()
plt.show()

fig = plt.gcf() 
fig.set_size_inches(10,10)
plt.contourf(L,A,Z_variance,250,cmap='turbo')
plt.colorbar()
plt.scatter(450,0.9,s=200,c='red',label='True theta: [450,0.9]')
plt.legend()


In [ ]:
fig = plt.gcf() 
fig.set_size_inches(15,10)
plt.tricontourf([path_[k][0][0] for k in range(len(path_))], [path_[k][0][1] for k in range(len(path_))], [path_[k][1] for k in range(len(path_))], levels=100, linewidths=0.5,cmap='viridis_r')
plt.colorbar()
plt.scatter([path_[k][0][0] for k in range(len(path_))], [path_[k][0][1] for k in range(len(path_))], c=[path_[k][1] for k in range(len(path_))],s=0.5,cmap='inferno')
plt.colorbar()
plt.title('simulated cov_mat with GNM data. true theta=(10000,1.7). 100 fixed locs,200 foxed days, Energy SR, evaluated over all data at each iter. Dual anni after 1000feval')
plt.show()

In [ ]:
idx_over6_5_SR = np.argwhere(np.array([path_[k][1] for k in range(len(path_))])<1.5).flatten()
fig = plt.gcf() 
fig.set_size_inches(15,10)
plt.tricontourf([path_[k][0][0] for k in idx_over6_5_SR], [path_[k][0][1] for k in idx_over6_5_SR], [path_[k][1] for k in idx_over6_5_SR], levels=100, linewidths=0.5,cmap='viridis_r')
plt.colorbar()
plt.scatter([path_[k][0][0] for k in idx_over6_5_SR], [path_[k][0][1] for k in idx_over6_5_SR], c=[path_[k][1] for k in idx_over6_5_SR],s=11,cmap='Oranges')
plt.colorbar()
plt.title('simulated cov_mat with GNM data. true theta=(10000,1.7). 100 fixed locs,200 foxed days, Energy SR, evaluated over all data at each iter. Dual anni after 1000feval. Only showing points where SR<6.45 =>99 points')
plt.show()

### Experiments for SR estimation

In [ ]:
subset_locations = []
def generate_subsample_locs(locs_nb):
    locs=[]
    while len(locs)<locs_nb:
        random_center = np.random.choice(range(len(GNM_landonly))) 
        locs = np.where([math.dist(GNM_landonly[k][5],GNM_landonly[random_center ][5])<(np.sqrt(locs_nb/np.pi)+3) for k in range(len(GNM_landonly))])[0]
    return np.random.choice(locs,size=locs_nb,replace=False)    
for s in range(50):
    subset_locations.append(generate_subsample_locs(locs_nb=200))

In [ ]:
def Spatial_Energy_Score_exp(theta,beta,sub_locs,days_nb,sim_nb,observations_x,censor_levels_di,dist_matrix,topo_matrix=None):
    '''
    theta: vector of 3 [Matern_param, coef_distmat, coef_topomat]. The coefs are applied to the matricies to form the ditance matrix used as input in the Matern kernel to form the covariance.
    sub_locs: a set of subsets of location indexes with the same number of locs in each.
    days_nb: how many days to include on the subsample.
    sim_nb: number of simulation draws to compare with original data in the score.
    observations_x: x, observed rainfall transformed to a Gaussian normal scale. This is the data that we try to emulate.
    censor_levels_di: d, probability of no rain (1-p) transformed to a normal scale. These are used to censor simulations (x') when comparingt to x.
    '''
    # Subsample locations. Choose a random subset the list of subsets - sub_locs.
    locs = sub_locs 
    locs_nb = len(locs)
    # Subsample days.
    days = np.random.choice(range(len(observations_x)),size=days_nb,replace=False)

    # Select relevant subsamples of data.
    x = observations_x[days,:]
    d = censor_levels_di[days,:]
    
    # Create covariance matrix.
    if np.sum(topo_matrix==None):
        cov_mat = np.nan_to_num(Matern(length_scale=theta,nu=3.5).__call__(dist_matrix[locs,:][:,locs]))
    else:
        cov_mat = np.nan_to_num(Matern(length_scale=theta,nu=3.5).__call__(0.9*dist_mat[locs,:][:,locs]+((0.1)/70)*topo_dist[locs,:][:,locs]))

    # Draw samples from the multivariate nromal with appropriate covariance.
    simulations = scs.multivariate_normal(mean=np.zeros(locs_nb),cov=cov_mat,allow_singular=True).rvs(size=sim_nb)
    
    # Censor simulated values and format them into [[sim0-day0,sim0-day1,...,sim0-dayn] ,..., [sim_m-day0,sim_m-day1,...,sim_m-dayn]]. Gives sim_nb*days_nb=m*n vectors of len=locs_nb. 
    censored_sims = np.maximum(np.repeat(simulations,days_nb,axis=0),np.tile(d,(sim_nb,1)))

    # ||x' - x' ||_2 component of energy score
    sims_by_day = [censored_sims[0+k::days_nb] for k in range(days_nb)] # regroup to get [[sim(1,day1), sim(2,day1),...,sim(m,day1)],...,[sim(1,dayn), sim(2,dayn),...,sim(m,dayn)]].
    
    sim_L2_diff = 0
    
    for sim_day_i in sims_by_day: # don't know how to make it more efficient... maybe can parallelize to compute complete score
        
        # ||x' - x' ||_2 component of energy score
            # OLD: einsum from Lorenzos code on each sim_day_i was 9x slower then doing pdist on each sim_day_i. Both give the same answer
        '''
        # matrix of all combinations over i of [vector_i-vector_1, ... , vector_i-vector_n]
        temp_diff_combinations_matrix = sim_day_i.reshape(1,sim_nb,locs_nb)-sim_day_i.reshape(sim_nb,1,locs_nb) 
        # einsum creates a matrix of sum of squares, then take sqrt to get L2.
        sim_L2_diff+=0.5*np.sum(np.sqrt(np.einsum('ijk, ijk -> ij',temp_diff_combinations_matrix,temp_diff_combinations_matrix))) 
        '''
            # New: pdist takes the offdiagonal of an L2 distance matrix, only counting once each combination of Y,Y'.
        sim_L2_diff+= 2 * np.sum(np.power(pdist(sim_day_i),beta))

    # Add both parts of Energy.
    Energy = 2 * np.mean(np.power(np.linalg.norm(censored_sims - np.tile(x,(sim_nb,1)),axis=1),beta)) - sim_L2_diff/(days_nb*sim_nb*(sim_nb-1))
    
    
    path_.append([theta,Energy])
    print([theta,Energy])

    return Energy

In [ ]:
# Generating fake data

# 1 - fixing a theta and generating data from that, then estimating to retrieve the same theta.
def generate_subsample_locs(location_number):
    l=[]
    while len(l)<location_number:
        random_center = np.random.choice(range(len(GNM_landonly))) 
        l = np.where([math.dist(GNM_landonly[k][5],GNM_landonly[random_center ][5])<(np.sqrt(location_number/np.pi)+2) for k in range(len(GNM_landonly))])[0]
    return np.random.choice(l,size=location_number,replace=False)   
 
'''
subset_locations = []
for s in range(250):
    subset_locations.append(generate_subsample_locs(location_number=800))
'''
fake_locs=generate_subsample_locs(400)
fake_theta = [450,0.9]
fake_cov_mat =np.nan_to_num(Matern(length_scale=fake_theta[0],nu=3.5).__call__(fake_theta[1]*dist_mat[fake_locs,:][:,fake_locs]+((1-fake_theta[1])/70)*topo_dist[fake_locs,:][:,fake_locs])) 
fake_data = scs.multivariate_normal(mean=np.zeros(len(fake_locs)),cov=fake_cov_mat,allow_singular=True).rvs(size=5000)
for day,day_data in enumerate(fake_data):
    fake_data[day]=np.maximum(day_data,[censor_levels_di[day][l] for l in fake_locs])
fake_censor_levels_di = censor_levels_di[range(5000),:][:,fake_locs]


In [ ]:
(0.9*dist_mat[fake_locs,:][:,fake_locs]+((1-0.9)/70)*topo_dist[fake_locs,:][:,fake_locs]).shape

In [ ]:
fig = plt.gcf() 
fig.set_size_inches(14,10)

plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(GNM_landonly))],[-GNM_landonly[loc][5][0] for loc in range(len(GNM_landonly))],c='grey',alpha=0.3,s=27,marker='s')
plt.scatter(GNM_landonly[locs[0]][5][1],
-GNM_landonly[locs[0]][5][0],
c='orange',label='Subset of locations',
cmap='inferno_r',s=100,marker='s')

plt.scatter([GNM_landonly[loc][5][1] for loc in locs],
[-GNM_landonly[loc][5][0] for loc in locs],
c=[GNM_landonly[loc][3][300] for loc in locs],
cmap='inferno_r',s=21,marker='s')
plt.legend()
plt.yticks(ticks=np.linspace(-100,0,5),labels=np.linspace(0,100,5))
plt.xlim(0,140)
plt.ylim(-100,0)
plt.savefig('sub_locs.eps', format='eps')
plt.show()

In [ ]:
fig = plt.gcf() 
fig.set_size_inches(14,10)

plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(GNM_landonly))],[-GNM_landonly[loc][5][0] for loc in range(len(GNM_landonly))],c='grey',alpha=0.3,s=27,marker='s')
plt.scatter(GNM_landonly[fake_locs[0]][5][1],
-GNM_landonly[fake_locs[0]][5][0],
c='orange',label='Subset of locations',
cmap='inferno_r',s=100,marker='s')

plt.scatter([GNM_landonly[loc][5][1] for loc in fake_locs],
[-GNM_landonly[loc][5][0] for loc in fake_locs],
c=[GNM_landonly[loc][3][300] for loc in fake_locs],
cmap='inferno_r',s=21,marker='s')
plt.legend()
plt.yticks(ticks=np.linspace(-100,0,5),labels=np.linspace(0,100,5))
plt.xlim(0,140)
plt.ylim(-100,0)

plt.show()

In [ ]:
# 100 sets of 800 locs, 700 days, 150 sims, beta=0.2 , avg of 4 =>25s/SRavg
path_=[]
Spatial_Energy_Score_exp(theta=450,
                                      beta=0.2, 
                                      sub_locs=fake_locs, 
                                      days_nb=700,sim_nb=150, 
                                      observations_x=fake_data, 
                                      censor_levels_di=fake_censor_levels_di, 
                                      dist_matrix=dist_mat,
                                      topo_matrix=topo_dist)

In [ ]:
qqq=[[],[]]
for a in tqdm(range(20)):
    qqq[1].append(Spatial_Energy_Score_sublocs(theta=[30000,3],beta=0.2,sub_locs=subset_locations,days_nb=700,sim_nb=100,observations_x=fake_data,censor_levels_di=censor_levels_di,dist_matrix=dist_mat,topo_matrix=topo_dist)
)
    qqq[0].append(Spatial_Energy_Score_sublocs(theta=[100,4],beta=0.2,sub_locs=subset_locations,days_nb=700,sim_nb=100,observations_x=fake_data,censor_levels_di=censor_levels_di,dist_matrix=dist_mat,topo_matrix=topo_dist)
)
plt.plot(qqq[0])
plt.plot(qqq[1],color='red')
np.mean(np.array(qqq),axis=1) # 750 locations 700 days 100 sims, beta = 0.2

In [ ]:
#path_ = []

dual_annealing(Spatial_Energy_Score_exp,[(100,1000)],args=(0.2,fake_locs,700,150,fake_data,fake_censor_levels_di,dist_mat,topo_dist),no_local_search=True,maxiter=13000)

#### Experiment with simple Gaussian spatial copula

In [ ]:
fake_locs=locs # got these from 'finding a local set of points' section
fake_theta = [450,0.9]
fake_cov_mat =np.nan_to_num(Matern(length_scale=fake_theta[0],nu=3.5).__call__(fake_theta[1]*dist_mat[fake_locs,:][:,fake_locs]+((1-fake_theta[1])/70)*topo_dist[fake_locs,:][:,fake_locs])) 
fake_data = scs.multivariate_normal(mean=np.zeros(len(fake_locs)),cov=fake_cov_mat,allow_singular=True).rvs(size=5000) #5000 samples

def Spatial_Energy_Score_exp_classic(theta,beta,sub_locs,days_nb,sim_nb,observations_x,dist_matrix,topo_matrix=None):
    '''
    theta: vector of 3 [Matern_param, coef_distmat, coef_topomat]. The coefs are applied to the matricies to form the ditance matrix used as input in the Matern kernel to form the covariance.
    sub_locs: a set of subsets of location indexes with the same number of locs in each.
    days_nb: how many days to include on the subsample.
    sim_nb: number of simulation draws to compare with original data in the score.
    observations_x: x, observed rainfall transformed to a Gaussian normal scale. This is the data that we try to emulate.
    censor_levels_di: d, probability of no rain (1-p) transformed to a normal scale. These are used to censor simulations (x') when comparingt to x.
    '''
    # Subsample locations. Choose a random subset the list of subsets - sub_locs.
    locs = sub_locs 
    locs_nb = len(locs)
    # Subsample days.
    days = np.random.choice(range(len(observations_x)),size=days_nb,replace=False)

    # Select relevant subsamples of data.
    x = observations_x[days,:]
    
    # Create covariance matrix.
    if np.sum(topo_matrix==None):
        cov_mat = np.nan_to_num(Matern(length_scale=theta,nu=3.5).__call__(dist_matrix[locs,:][:,locs]))
    else:
        cov_mat = np.nan_to_num(Matern(length_scale=theta,nu=3.5).__call__(0.9*dist_mat[locs,:][:,locs]+((0.1)/70)*topo_dist[locs,:][:,locs]))

    # Draw samples from the multivariate nromal with appropriate covariance.
    simulations = scs.multivariate_normal(mean=np.zeros(locs_nb),cov=cov_mat,allow_singular=True).rvs(size=sim_nb)
    
    censored_sims = np.repeat(simulations,days_nb,axis=0)

    # ||x' - x' ||_2 component of energy score
    sims_by_day = [censored_sims[0+k::days_nb] for k in range(days_nb)] # regroup to get [[sim(1,day1), sim(2,day1),...,sim(m,day1)],...,[sim(1,dayn), sim(2,dayn),...,sim(m,dayn)]].
    
    sim_L2_diff = 0
    
    for sim_day_i in sims_by_day: # don't know how to make it more efficient... maybe can parallelize to compute complete score
        
        # ||x' - x' ||_2 component of energy score
            # OLD: einsum from Lorenzos code on each sim_day_i was 9x slower then doing pdist on each sim_day_i. Both give the same answer
        '''
        # matrix of all combinations over i of [vector_i-vector_1, ... , vector_i-vector_n]
        temp_diff_combinations_matrix = sim_day_i.reshape(1,sim_nb,locs_nb)-sim_day_i.reshape(sim_nb,1,locs_nb) 
        # einsum creates a matrix of sum of squares, then take sqrt to get L2.
        sim_L2_diff+=0.5*np.sum(np.sqrt(np.einsum('ijk, ijk -> ij',temp_diff_combinations_matrix,temp_diff_combinations_matrix))) 
        '''
            # New: pdist takes the offdiagonal of an L2 distance matrix, only counting once each combination of Y,Y'.
        sim_L2_diff+= 2 * np.sum(np.power(pdist(sim_day_i),beta))
    # Add both parts of Energy.
    Energy = 2 * np.mean(np.power(np.linalg.norm(censored_sims - np.tile(x,(sim_nb,1)),axis=1),beta)) - sim_L2_diff/(days_nb*sim_nb*(sim_nb-1))
    
    
    path_.append([theta,Energy])
    print([theta,Energy])

    return Energy

In [ ]:
[Spatial_Energy_Score_exp_classic(theta=200,beta=0.5,sub_locs=fake_locs,days_nb=3000,sim_nb=150,observations_x=fake_data,dist_matrix=dist_mat,topo_matrix=topo_dist) for k in range(10)]

In [ ]:
path_ = []
simple_gaussian_experiment = []
for theta_val in tqdm(np.linspace(200,1000,500)):

    avg = np.mean([Spatial_Energy_Score_exp_classic(theta=theta_val,beta=0.5,sub_locs=fake_locs,days_nb=3000,sim_nb=150,observations_x=fake_data,dist_matrix=dist_mat,topo_matrix=topo_dist) for i in range(15)])
    simple_gaussian_experiment.append(avg)  


In [ ]:
simple_gaussian_experiment

In [ ]:
path_ = np.array(path_)
plt.scatter(path_[:,0],path_[:,1])
plt.plot(np.linspace(200,1000,500),simple_gaussian_experiment,color='red')
min_idx=np.where(simple_gaussian_experiment==np.min(simple_gaussian_experiment))[0]

plt.plot([np.linspace(200,1000,500)[min_idx],np.linspace(200,1000,500)[min_idx]],[4.4,4.9],color='black',linestyle='dotted',linewidth=3,label=r'Estimated $\theta ^{*}$ = '+str(np.linspace(200,1000,500)[min_idx]))
plt.legend()

In [ ]:
for k in range(30):
    sr_avg_scores = [np.mean([simple_gaussian_experiment[i:i+k]]) for i in range(len(simple_gaussian_experiment))]
    print(k,np.linspace(200, 1000, 500)[np.where(sr_avg_scores==np.min(sr_avg_scores))])

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 30})
fig = plt.gcf() 
fig.set_size_inches(15,10)


#plt.scatter( np.linspace(200,1000,500) , simple_gaussian_experiment ,color='orange',s=30,label='Energy Score Estimates')
plt.scatter(np.linspace(200,1000,495) , [np.mean([simple_gaussian_experiment[i:i+10]]) for i in range(len(simple_gaussian_experiment)-5)],color='orange' ,s=30,label=' Energy Score Estimates')
plt.plot([450,450],[4.61,4.69],color='black',linestyle='dotted',linewidth=3,label=r'True $\theta ^{*}$ = 450')

plt.legend()
plt.tight_layout()
plt.xlabel(r'$\theta$')
plt.ylabel('Energy Score')
plt.savefig('SR_lscale_fakedata_simple_gauss2.eps', format='eps')
plt.show()


In [ ]:
with open('path_ex_simple_gaussian_attempt2.txt','wb') as f:
    pickle.dump([path_,simple_gaussian_experiment],f)

### Simulation

In [ ]:
#cov_rain = np.cov(np.transpose(obs_norm_all))
#sim_cov_rain = scs.multivariate_normal(mean=np.zeros(len(dist_mat)),cov=cov_rain).rvs(size=25)
theta = [350,0.9] 
sim_test = scs.multivariate_normal(mean=np.zeros(len(dist_mat)),cov= np.nan_to_num(Matern(length_scale=theta[0],nu=3.5).__call__(theta[1]*dist_mat+((1-theta[1])/70)*topo_dist)) ,allow_singular=True).rvs(size=100)

#theta2 = [310, 1] 
#sim_test2 = scs.multivariate_normal(mean=np.zeros(len(dist_mat)),cov= np.nan_to_num(Matern(length_scale=theta2[0],nu=3.5).__call__(theta[1]*dist_mat+((1-theta[1])/70)*topo_dist)) ,allow_singular=True).rvs(size=25)

In [ ]:
def GNM_ppf(x_one_day_all_locs,day_idx):
    m = np.array([GNM_landonly[l][0][day_idx] for l in range(len(x_one_day_all_locs))])
    d = np.array([GNM_landonly[l][1][day_idx] for l in range(len(x_one_day_all_locs))])
    p = np.array([GNM_landonly[l][4][day_idx] for l in range(len(x_one_day_all_locs))])
    sim_u = scs.norm.cdf(x_one_day_all_locs)
    censored_u = (sim_u+p-1)/p
    return np.nan_to_num(scs.gamma(a=1/d,scale=m*d).ppf(censored_u),nan=0)
    

#GNM_landonly[location][param][day]


In [ ]:
#plot ampled normals_uncesored

day_index = 320

fig = plt.gcf() 
fig.set_size_inches(14,10)

plt.xlim(0,140)
plt.ylim(-100,0)
plt.yticks([])
plt.xticks([])
plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],[-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],c=[GNM_landonly[l][3][day_index] for l in range(len(GNM_landonly))][:len(sim_test[0])],cmap='inferno_r',s=27,marker='s')
plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
plt.title('Observed rain')
plt.show()

min_rain = min([GNM_landonly[l][3][day_index] for l in range(len(GNM_landonly))])
max_rain = max([GNM_landonly[l][3][day_index] for l in range(len(GNM_landonly))])

fig = plt.gcf() 
fig.set_size_inches(14,10)

plt.xlim(0,140)
plt.ylim(-100,0)
plt.yticks([])
plt.xticks([])
plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],[-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],c=np.median(np.array([GNM_ppf(sim,day_index) for sim in sim_test]),axis=0),cmap='inferno_r',s=27,marker='s',vmin=min_rain,vmax=max_rain)
plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
plt.title('Median simulated')
plt.show()

for idx,sim in enumerate(sim_test[:10]):
    '''fig = plt.gcf() 
    fig.set_size_inches(14,10)

    plt.xlim(0,140)
    plt.ylim(-100,0)
    plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],[-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],c=GNM_ppf(sim_test2[idx][:len(sim_test[0])],day_index),cmap='inferno_r',s=27,marker='s')
    plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
    plt.title(' test 2')
    plt.show()'''

    plt.xlim(0,140)
    plt.ylim(-100,0)
    fig = plt.gcf() 
    fig.set_size_inches(14,10)
    plt.yticks([])
    plt.xticks([])
    plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],[-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],c=GNM_ppf(sim,day_index),cmap='inferno_r',s=27,marker='s',vmin=min_rain,vmax=max_rain)
    plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
    plt.title('Simulated rain with distance only')
    plt.show()

In [ ]:
#Analysing biases introduces by copula sampling against GNM marginals.

true_median = scs.gamma(a=np.array([1/GNM_landonly[l][1][140] for l in range(len(GNM_landonly))]),
          scale=np.multiply( np.array([GNM_landonly[l][0][140] for l in range(len(GNM_landonly))]),np.array([GNM_landonly[l][1][140] for l in range(len(GNM_landonly))]) )).ppf(
    np.divide((np.array([GNM_landonly[l][4][140] for l in range(len(GNM_landonly))])-0.5),np.array([GNM_landonly[l][4][140] for l in range(len(GNM_landonly))]))
          )
sim_median = np.mean(np.array([GNM_ppf(sim_test[s],140) for s in range(len(sim_test))]),axis=0)

plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],[-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],c=[GNM_landonly[loc][3][140] for loc in range(len(sim_test[0]))],cmap='inferno_r',s=2.8,marker='s')
plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
plt.title('true rain')
plt.xlim(0,140)
plt.ylim(-100,0)
plt.show()
min_rain = np.min([GNM_landonly[loc][3][140] for loc in range(len(sim_test[0]))])
max_rain = np.max([GNM_landonly[loc][3][140] for loc in range(len(sim_test[0]))])

plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],[-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],c=[GNM_landonly[loc][0][140] for loc in range(len(sim_test[0]))],cmap='inferno_r',s=2.8,marker='s',vmin=min_rain,vmax=max_rain)
plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
plt.title('GNM mean')
plt.xlim(0,140)
plt.ylim(-100,0)
plt.show()
plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],[-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],c=true_median,cmap='inferno_r',s=2.7,marker='s',vmin=min_rain,vmax=max_rain)
plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
plt.title('GNM_median')
plt.xlim(0,140)
plt.ylim(-100,0)
plt.show()

plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],[-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],c=sim_median,cmap='inferno_r',s=2.7,marker='s',vmin=min_rain,vmax=max_rain)
plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
plt.title('sim_median')
plt.xlim(0,140)
plt.ylim(-100,0)
plt.show()
fig = plt.gcf() 
fig.set_size_inches(14,10)
max_dif = np.max(np.abs(true_median-sim_median))
plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],[-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],c=true_median-sim_median,cmap='RdBu',s=27,marker='s',vmin=-max_dif,vmax=max_dif)
plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
plt.title('GNM_median-sim_median')
plt.xlim(0,140)
plt.ylim(-100,0)
plt.show()

fig = plt.gcf() 
fig.set_size_inches(14,10)
max_dif = np.max(np.abs([GNM_landonly[loc][0][140] for loc in range(len(sim_test[0]))]-sim_median))
plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],[-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],c=[GNM_landonly[loc][0][140] for loc in range(len(sim_test[0]))]-sim_median,cmap='RdBu',s=27,marker='s',vmin=-max_dif,vmax=max_dif)
plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
plt.title('GNM_mean-sim_median')
plt.xlim(0,140)
plt.ylim(-100,0)
plt.show()

fig = plt.gcf() 
fig.set_size_inches(14,10)
max_dif = np.max(np.abs([GNM_landonly[loc][3][140] for loc in range(len(sim_test[0]))]-sim_median))
plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],[-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],c=[GNM_landonly[loc][3][140] for loc in range(len(sim_test[0]))]-sim_median,cmap='RdBu',s=27,marker='s',vmin=-max_dif,vmax=max_dif)
plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
plt.title('true_rain-sim_median')
plt.xlim(0,140)
plt.ylim(-100,0)
plt.show()

### Covariance matrix testing

In [ ]:
cov_rain = np.cov(np.transpose(obs_norm_all))
sim_cov_rain = scs.multivariate_normal(mean=np.zeros(len(dist_mat)),cov=cov_rain).rvs(size=25)

In [ ]:
plt.matshow(dist_mat)
plt.colorbar()
plt.matshow(topo_dist)#
plt.colorbar()

In [ ]:
lim_mat = [0,4087]
theta = [60,0.045,1/1500]
# SR [1.53735034e+02,3.5, 5.99074130e+03,3.5 ] #MLE:[208.33370545627545,3.5, 4500.88496935951,3.5]

plt.matshow(cov_rain[lim_mat[0]:lim_mat[1],lim_mat[0]:lim_mat[1]],cmap='viridis_r',vmin=0)
plt.colorbar()
plt.title('cov')


topo_matern = np.nan_to_num(Matern(length_scale=12000,nu=3.5).__call__(topo_dist[lim_mat[0]:lim_mat[1],lim_mat[0]:lim_mat[1]]))
dist_matern= np.nan_to_num(Matern(length_scale=600,nu=3.5).__call__(dist_mat[lim_mat[0]:lim_mat[1],lim_mat[0]:lim_mat[1]]))
cov_matern_add = 0.8* np.nan_to_num(Matern(length_scale=theta[0],nu=3.5).__call__(theta[1]*dist_mat[lim_mat[0]:lim_mat[1],lim_mat[0]:lim_mat[1]]+theta[2]*topo_dist[lim_mat[0]:lim_mat[1],lim_mat[0]:lim_mat[1]]))
plt.matshow(cov_matern_add,cmap='viridis_r',vmin=0,vmax=1.1)
plt.colorbar()
plt.title('Adding distances before Matern kernel')

plt.matshow(dist_matern,cmap='viridis_r',vmin=0)
plt.title('distance matern')
plt.colorbar()

plt.matshow(topo_matern,cmap='viridis_r',vmin=0)
plt.title('topo matern')
plt.colorbar()


In [ ]:
lim_mat = [0,2000]

plt.matshow(Matern(length_scale=9000,nu=3.5).__call__(dist_mat[lim_mat[0]:lim_mat[1],lim_mat[0]:lim_mat[1]]),cmap='viridis_r',vmin=0)
plt.colorbar()
plt.title('cov')

In [ ]:
plt.matshow(cov_rain[lim_mat[0]:lim_mat[1],lim_mat[0]:lim_mat[1]],cmap='viridis_r',vmin=0)
plt.colorbar()
plt.title('cov')

### Working with elevation

In [ ]:
import netCDF4 as nc
fn = 'topo_0.1_degree.nc'
ds = nc.Dataset(fn)
h = ds['Band1'][:]
h = np.flip(np.matrix(h),axis=0)
topo_dist = euclidean_distances(np.array([h[loc[5][0],loc[5][1]+2] for loc in GNM_landonly]).reshape(-1,1))

In [ ]:
plt.matshow(topo_dist)
plt.show()
plt.matshow(h)